The Following Model(s) uses available dataset @ kaggle & github (ref: https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv). The mentioned dataset has the following columns:

*   age
*   sex
*   bmi
*   children
*   smoker
*   region
*   charges

Using Neural Network Regression, we hope that our model will be able to learn how to predict the dependent variable (charges) using available features (age,sex,bmi,children,smoker,region).

In [23]:
# import the needed libraroes 
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [24]:
insurance_dataset = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")

In [25]:
insurance_dataset

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


some indicators (sex, smoker, region) are of object datatype. we use one-hot-encoding to be able to use them as numerical values

In [26]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

ct = make_column_transformer(
    (MinMaxScaler(), ["age", "bmi", "children"]), # get all values between 0 and 1 (Scalling)
    (OneHotEncoder(handle_unknown="ignore"), ["sex", "smoker", "region"])
)

x = insurance_dataset.drop("charges", axis=1)
y = insurance_dataset["charges"]

print(x)

      age     sex     bmi  children smoker     region
0      19  female  27.900         0    yes  southwest
1      18    male  33.770         1     no  southeast
2      28    male  33.000         3     no  southeast
3      33    male  22.705         0     no  northwest
4      32    male  28.880         0     no  northwest
...   ...     ...     ...       ...    ...        ...
1333   50    male  30.970         3     no  northwest
1334   18  female  31.920         0     no  northeast
1335   18  female  36.850         0     no  southeast
1336   21  female  25.800         0     no  southwest
1337   61  female  29.070         0    yes  northwest

[1338 rows x 6 columns]


In [27]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [28]:
ct.fit(x_train)

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 ['age', 'bmi', 'children']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['sex', 'smoker', 'region'])])

In [29]:
x_train.loc[0]

age                19
sex            female
bmi              27.9
children            0
smoker            yes
region      southwest
Name: 0, dtype: object

In [30]:
x_train_normal = ct.transform(x_train)
x_test_normal = ct.transform(x_test)

In [31]:
x_train_normal[0]

array([0.60869565, 0.10734463, 0.4       , 1.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 0.        ,
       0.        ])

In [33]:
tf.random.set_seed(42)

imodel = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(50),
  tf.keras.layers.Dense(1)
])

imodel.compile(loss=tf.keras.losses.mae,
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['mae'])

imodel.fit(x_train_normal, y_train, epochs=400)

Epoch 1/400
34/34 [==============================] - 1s 2ms/step - loss: 13341.0850 - mae: 13341.0850
Epoch 2/400
34/34 [==============================] - 0s 2ms/step - loss: 13317.0459 - mae: 13317.0459
Epoch 3/400
34/34 [==============================] - 0s 1ms/step - loss: 13241.4385 - mae: 13241.4385
Epoch 4/400
34/34 [==============================] - 0s 1ms/step - loss: 13066.5664 - mae: 13066.5664
Epoch 5/400
34/34 [==============================] - 0s 1ms/step - loss: 12740.6562 - mae: 12740.6562
Epoch 6/400
34/34 [==============================] - 0s 1ms/step - loss: 12210.2100 - mae: 12210.2100
Epoch 7/400
34/34 [==============================] - 0s 1ms/step - loss: 11453.6846 - mae: 11453.6846
Epoch 8/400
34/34 [==============================] - 0s 1ms/step - loss: 10584.8350 - mae: 10584.8350
Epoch 9/400
34/34 [==============================] - 0s 1ms/step - loss: 9719.1953 - mae: 9719.1953
Epoch 10/400
34/34 [==============================] - 0s 2ms/step - loss: 8963.6152 

In [34]:
imodel.evaluate(x_test_normal, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 3159.0046 - mae: 3159.0046


[3159.004638671875, 3159.004638671875]